# IMPORTANDO BIBLIOTECAS

In [ ]:
# Importando bibliotecas necessárias
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# EXIBIÇÃO DOS DATAFRAMES

In [ ]:
# Configurando a exibição do dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

# LEITURA DOS ARQUIVOS

In [ ]:
# Lista com caminhos locais distintos
diretorios_locais = [
    r'C:\Users\brand\OneDrive\Área de Trabalho\Fraud_Detection_ML\Data\\',
    r'C:\Users\Giovanni Maia\Desktop\Dados\\'
]

diretorio_github= 'https://media.githubusercontent.com/media/jeanbrandao01/Fraud_Detection_ML/dev/Data/'

def carregar_dataframes(diretorios_locais, diretorio_github):
    for diretorio_local in diretorios_locais:
        try:
            # Tente carregar o DataFrame de fraudTest.csv a partir do diretório local
            df_test = pd.read_csv(diretorio_local + 'fraudTest.csv')

            # Tente carregar o DataFrame de fraudTrain.csv a partir do diretório local
            df_train = pd.read_csv(diretorio_local + 'fraudTrain.csv')

            # Se o carregamento for bem-sucedido, retorne os DataFrames
            return df_test, df_train
        except FileNotFoundError:
            continue  # Continue para o próximo diretório local em caso de erro

    # Se nenhum diretório local contiver os arquivos, carregue do GitHub
    url_test = diretorio_github + 'fraudTest.csv'
    url_train = diretorio_github + 'fraudTrain.csv'

    df_test = pd.read_csv(url_test)
    df_train = pd.read_csv(url_train)

    return df_test, df_train

df_test, df_train = carregar_dataframes(diretorios_locais,diretorio_github)


# EXIBIÇÃO DOS DATAFRAMES ORIGINAIS

In [ ]:
# Dataframe "fraudTest.csv"
df_test.head()

In [ ]:
# Dataframe "fraudTrain.csv"
df_train.head()

# ANÁLISE DESCRITIVA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [ ]:
# Algumas estatisticas do dataset
df_train.describe()

In [ ]:
# Verificando as colunas existentes
df_train.info()

In [ ]:
# Realizando a contagem de valores da coluna "merchant"
df_train.merchant.value_counts()

In [ ]:
# Realizando a contagem de valores da coluna "is_fraud"
print("Contagem de valores da coluna is_fraud:")
print("-"*39)
print(df_train.is_fraud.value_counts())


In [ ]:
# Verificando a extensão do dataset
print("Extensão do dataset:")
print("-"*23)
print(df_train.shape)


In [ ]:
# Verificando os nomes de cada coluna 
print("Colunas do dataset")
print(df_train.columns)

# ANÁLISE EXPLORATÓRIA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [ ]:
# Calcular a proporção de fraudes
fraud_proportion = df_train['is_fraud'].value_counts(normalize=True)

# Criar um DataFrame para facilitar a criação do gráfico
fraud_data = pd.DataFrame({'Classe de Fraude': fraud_proportion.index, 'Proporção': fraud_proportion.values})

# Criar o gráfico de barras com o Plotly
fig = px.bar(fraud_data, x='Classe de Fraude', y='Proporção', text='Proporção', title='Distribuição de Fraudes', labels={'Classe de Fraude': 'Classe de Fraude'})
fig.update_traces(texttemplate='%{text:.2%}', textposition='outside')
fig.show()



In [ ]:
# mostrando a porcentagem de transações fraudulentas e não fraudulentas para cada categoria presente na coluna category 
round(pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index')*100,2)

In [ ]:
# Cria uma tabela de contingência
crosstab = pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index') * 100

# Reinicia o índice para facilitar o manuseio
crosstab = crosstab.reset_index()

# Cria um gráfico de barras empilhadas usando o Plotly Express
fig = px.bar(crosstab, x='category', y=[0, 1], title="Gráfico de Barras Empilhadas da Tabela de Contingência (Percentual)",
             labels={0: "Não Fraude", 1: "Fraude"},
             height=400)

# Atualiza o layout para um gráfico de barras empilhadas
fig.update_layout(barmode='stack')

# Mostra o gráfico
fig.show()


In [ ]:
# Agrupa os dados por estado e calcula o número total de transações em cada estado
total_transacoes_por_estado = df_train["state"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por estado e calcula o número de transações fraudulentas em cada estado
transacoes_fraudulentas_por_estado = dados_fraudulentos["state"].value_counts()

# Calcula a taxa de fraude para cada estado (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_estado / total_transacoes_por_estado).fillna(0)

# Reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude.reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Retorna as últimas 20 linhas (estados) dessa Series, representando os estados com o menor número de transações. 
total_transacoes_por_estado.tail(20)

## Fraudes por cidade

In [ ]:
# Retorna o número de cidades únicas presentes na coluna cidade
df_train.city.nunique()

In [ ]:
# Agrupa os dados por cidade e calcula o número total de transações em cada cidade
total_transacoes_por_cidade = df_train["city"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por cidade e calcula o número de transações fraudulentas em cada cidade
transacoes_fraudulentas_por_cidade = dados_fraudulentos["city"].value_counts()

# Calcula a taxa de fraude para cada cidade (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_cidade / total_transacoes_por_cidade).fillna(0)

# Seleciona as cidades com taxa de fraude igual a 1, reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude[taxa_de_fraude == 1].reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um violin plot usando o Seaborn para valor da compra por classe
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.violinplot(data=df_train.query('amt <= 200'), x='is_fraud', y='amt')
# Define o título e rótulos
plt.title('Violin Plot - Distribuição do Valor da Compra por Classe de Fraude')
plt.xlabel('Classe de Fraude')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um boxplot usando o Seaborn para distruibuição do valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.boxplot(data=df_train.query('amt <= 200'), y='amt', color='skyblue')  # Adicionado cor para melhor visualização

# Define o título e rótulos
plt.title('Boxplot - Distribuição do Valor da Compra')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Escolhe uma paleta de cores para as barras (cores mais claras)
bar_palette = sns.color_palette("Blues", as_cmap=True)

# Escolhe uma cor mais escura para a linha do KDE
line_color = 'navy'

# Cria um histograma com KDE usando o Seaborn para valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt <= 200'), x='amt', kde=True)
sns.kdeplot(data=df_train.query('amt <= 200')['amt'], color=line_color, linewidth=2)

# Define o título e rótulos
plt.title('Histograma com KDE - Valor da Compra')
plt.xlabel('Valor da Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um histograma com KDE usando o Seaborn para valores de compra acima de 2000
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt > 2000'), x='amt', kde=True, color='red')  # Adiciona cor para melhor visualização

# Define o título e rótulos
plt.title('Histograma com KDE - Valor de Compra Acima de 2000')
plt.xlabel('Valor de Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Selecionar apenas as colunas numéricas
numeric_columns = df_train.select_dtypes(include=['int64', 'float64'])

# Calculando a matriz de correlação para colunas numéricas
correlation_matrix = numeric_columns.corr()

# Criando uma figura maior
plt.figure(figsize=(20, 10))

# Criando um mapa de calor de correlação com paleta de cores personalizada
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', center=0, cbar_kws={'label': 'Correlação'})
plt.title('Mapa de Calor de Correlação')

plt.show()


In [ ]:
# Análise temporal

# Convertendo a coluna 'trans_date_trans_time' para um objeto de data e hora
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])

# Extraindo características de data e hora
df_train['ano'] = df_train['trans_date_trans_time'].dt.year
df_train['mes'] = df_train['trans_date_trans_time'].dt.month
df_train['dia_da_semana'] = df_train['trans_date_trans_time'].dt.day_name()
df_train['hora'] = df_train['trans_date_trans_time'].dt.hour
df_train['minuto'] = df_train['trans_date_trans_time'].dt.minute
df_train['segundo'] = df_train['trans_date_trans_time'].dt.second

# Filtrando fraudes
frauds = df_train[df_train['is_fraud'] == 1]

# Contando fraudes por mês
frauds_by_month = frauds['mes'].value_counts().sort_index().reset_index()
frauds_by_month.columns = ['Mês', 'Número de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(frauds_by_month, x='Mês', y='Número de Fraudes', title='Fraudes por Mês',
             labels={'Mês': 'Mês', 'Número de Fraudes': 'Número de Fraudes'})

# Adicionando números em cima das barras
fig.update_traces(text=frauds_by_month['Número de Fraudes'], textposition='outside')

fig.show()


In [ ]:
# Contagem de ocorrências por categoria
category_counts = df_train['category'].value_counts().reset_index()
category_counts.columns = ['Categoria', 'Contagem']

# Agrupando por categoria e calculando a proporção de fraudes
fraud_proportion_by_category = df_train.groupby('category')['is_fraud'].mean().reset_index()
fraud_proportion_by_category.columns = ['Categoria', 'Proporção de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(fraud_proportion_by_category, x='Categoria', y='Proporção de Fraudes', title='Proporção de Fraudes por Categoria',
             labels={'Categoria': 'Categoria', 'Proporção de Fraudes': 'Proporção de Fraudes'})

# Formatando os números com cinco casas decimais
fraud_proportion_by_category['Proporção de Fraudes'] = fraud_proportion_by_category['Proporção de Fraudes'].round(5)

# Adicionando os números formatados (com cinco casas decimais) em cima das barras
fig.update_traces(text=fraud_proportion_by_category['Proporção de Fraudes'], textposition='outside')

fig.show()


# FEATURE ENGINEERING

In [ ]:
df_train.head()

In [ ]:
# Preparação dos dados e divisão em treino/teste
features = df_train[['amt', 'category', 'gender', 'city', 'state', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'dia_da_semana', 'hora', 'minuto', 'segundo']]
labels = df_train['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)


In [ ]:
# Definição de transformadores para Pré-processamento
numericas_continuas = ['amt', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'hora', 'minuto', 'segundo']
string_categoricas = ['category', 'gender', 'city', 'state', 'dia_da_semana']

# PIPELINE

In [ ]:

# Transformadores para variáveis numéricas contínuas
numeric_transformer = StandardScaler()

# Transformadores para variáveis categóricas
categorical_transformer = OneHotEncoder()

# Lista de tuplas indicando os transformadores a serem aplicados a cada conjunto de features
transformers = [
    ('num_continuas', numeric_transformer, numericas_continuas),
    ('str_categoricas', categorical_transformer, string_categoricas)
]

# Criação do ColumnTransformer integrado ao Pipeline
preprocessor = ColumnTransformer(transformers=transformers)


# TREINAMENTO DO MODELO

In [ ]:
# Treinamento do pré-processador
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


# CLASSIFICADORES

In [ ]:
# Lista de classificadores
classifiers_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]


In [ ]:
# Função para treinar, avaliar e prever para cada classificador
def train_evaluate_and_predict(classifier, X_train_transformed, y_train, X_test_transformed, y_test):
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_transformed, y_train)
    y_pred = pipe.predict(X_test_transformed)
    
    accuracy = accuracy_score(y_test, y_pred)   
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    result = {
        "Classifier": classifier.__class__.__name__,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Predictions": y_pred
    }

    # Adicionar métricas sem as previsões
    result_without_predictions = result.copy()
    result_without_predictions.pop("Predictions", None)

    return result, result_without_predictions

# Execução em paralelo para todos os classificadores
results, results_without_predictions = zip(*Parallel(n_jobs=-1)(
    delayed(train_evaluate_and_predict)(
        classifier, X_train_transformed, y_train, X_test_transformed, y_test
    ) for classifier in classifiers_list
))

# MÉTRICAS

In [ ]:
# Exiba os resultados sem as previsões
print("\nResults without Predictions:")
for result in results_without_predictions:
    print(result)
    print("---")  # Adiciona uma linha separadora entre os resultados de diferentes classificadores


In [ ]:
# Escolha o classificador com a maior F1-score (média harmônica de precisão e recall)
best_classifier_f1 = max(results_without_predictions, key=lambda x: (2 * x['Precision'] * x['Recall']) / (x['Precision'] + x['Recall']))

# Exiba o resultado
print("Melhor classificador (baseado no F1-score):")
print(best_classifier_f1)


In [ ]:
# Defina pesos para as métricas (pode ajustar conforme necessário)
weights = {
    'Accuracy': 1,
    'Precision': 1,
    'Recall': 1,
    'F1-score': 1
}

# Crie uma lista para armazenar as pontuações agregadas de cada classificador
scores = []

# Calcule as pontuações agregadas para cada classificador
for result in results_without_predictions:
    classifier_name = result['Classifier']
    score = sum(result[metric] * weights[metric] for metric in weights) / sum(weights.values())
    scores.append({'Classifier': classifier_name, 'Score': score})

# Encontre o classificador com a pontuação mais alta
best_classifier = max(scores, key=lambda x: x['Score'])

# Exiba o resultado
print("Melhor classificador (baseado na média ponderada das métricas):")
print(best_classifier)


# CROSS VALIDATION 

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Criar o pipeline com o classificador
classifier_cv = RandomForestClassifier()

# Criar o pipeline com o classificador
pipe_cv = Pipeline(steps=[('classifier', classifier_cv)])

# Definir a estratégia de validação cruzada (Stratified K-Fold)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definir as métricas desejadas
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

# Executar a validação cruzada e calcular as métricas
cv_results = cross_validate(pipe_cv, X_train_transformed, y_train, cv=stratified_kfold, scoring=scoring_metrics)

# Exibir os resultados
for metric in scoring_metrics:
    print(f"{metric.capitalize()} médio na validação cruzada:", cv_results[f'test_{metric}'].mean())
    print(f"Desvio padrão dos {metric.capitalize()} na validação cruzada:", cv_results[f'test_{metric}'].std())
    print("---")


# MATRIZ DE CONFUSÃO PARA TODOS OS INDICADORES (Não usei a cv pra esse conjunto, apenas a validação normal)

In [ ]:
# Lista de todas as colunas de previsão
prediction_columns = [
    'LogisticRegression_Prediction',
    'DecisionTreeClassifier_Prediction',
    'RandomForestClassifier_Prediction',
    'AdaBoostClassifier_Prediction',
    'GradientBoostingClassifier_Prediction'
]

# Loop para criar matrizes de confusão interativas para cada classificador
for classifier_column in prediction_columns:
    # Criar a matriz de confusão
    conf_matrix = confusion_matrix(df_predictions['is_fraud'], df_predictions[classifier_column])

    # Configurações do heatmap
    heatmap = ff.create_annotated_heatmap(
        z=conf_matrix,
        x=['Not Fraud', 'Fraud'],
        y=['Not Fraud', 'Fraud'],
        colorscale='Blues'
    )

    # Atualizar layout para adicionar rótulos
    heatmap.update_layout(
        xaxis=dict(title='Predicted'),
        yaxis=dict(title='True'),
        title=f'Confusion Matrix: is_fraud vs. {classifier_column}'
    )

    # Exibir o gráfico
    heatmap.show()


# DATA FRAME FINAL (AQUI É UM DF QUE TEM OS DADOS NORMAIS E OS DADOS DE PREDIÇÃO DOS 5 CLASSIFICADORES (Nome e predição)) 

In [ ]:
# Criação do DataFrame final
df_predictions = pd.DataFrame(df_train)  # Copiando o DataFrame original

# Adição das previsões ao DataFrame final
for result in results:
    classifier_name = result["Classifier"]
    predictions = result["Predictions"]
    
    # Ajuste para garantir que o número de previsões corresponda ao número de linhas no DataFrame original
    if len(predictions) != len(df_predictions):
        predictions = [0] * (len(df_predictions) - len(predictions)) + list(predictions)

    df_predictions[f"{classifier_name}_Prediction"] = predictions


# SALVANDO O ARQUIVO JÁ TRATADO

In [ ]:
# DataFrame
df_predictions.to_csv(
    'V.Final.csv',  # Caminho e nome do arquivo
    index=False,  # Não incluir índices no arquivo CSV
    encoding='utf-8',  # Codificação do arquivo (pode ser 'utf-8', 'latin1', etc.)
    sep=',',  # Delimitador de colunas (pode ser ',', ';', '\t', etc.)
    header=True  # Incluir cabeçalho no arquivo CSV
)
